In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(database="spezialthema", user="kevin", password="12345678")

In [3]:
with conn:
    with conn.cursor() as cur:
        cur.execute('select hospital_name from hospitals group by hospital_name HAVING COUNT (DISTINCT city) > 1;')
        print(cur.fetchall())
        
        

[('Blue Cross Clinic',), ('Imagery Health',), ('Open Clinic',), ('Van Holsen Community Hospital',)]


In [4]:
def create():
    with conn:
        with conn.cursor() as cur:
            cur.execute('CREATE TABLE test (A varchar(2), B varchar(2), C varchar(2),D varchar(2))')
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a1', 'b1', 'c1', 'd2'])
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a2', 'b2', 'c1', 'd1'])
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a1', 'b1', 'c2', 'd1'])

In [5]:
create()

DuplicateTable: relation "test" already exists


In [3]:
sql = pd.read_sql_query ('select * from test', conn)
df = pd.DataFrame(sql)
print(df)

    a   b   c   d
0  a1  b1  c1  d2
1  a2  b2  c1  d1
2  a1  b1  c2  d1


In [20]:
def sql(sql: str):
    return pd.read_sql(sql, conn)

with conn:
    with conn.cursor() as cur:
        cur.execute('EXPLAIN select * from test')
        print(cur.fetchall())
        cur.execute('SELECT * from test T1, test T2 where T1.A=T2.A and T1.B <> T2.B')
        print(cur.fetchall())
        cur.execute('SELECT * from test T1, test T2 where T1.A=T2.A and T1.C <> T2.C')
        print(cur.fetchall())
        cur.execute('select A from test group by A HAVING COUNT (DISTINCT B) > 1;')
        print(cur.fetchall())
        cur.execute('select A from test group by A HAVING COUNT (DISTINCT C) > 1;')
        print(cur.fetchall())
        
        #cur.execute('CREATE STATISTICS s6 (dependencies) ON a,b,c FROM test')
        cur.execute('ANALYZE test')   
        print(sql('SELECT dependencies FROM pg_stats_ext').loc[[5]])
        cur.execute('SELECT dependencies FROM pg_stats_ext')

        print(cur.fetchall())


[('Seq Scan on test  (cost=0.00..1.03 rows=3 width=12)',)]
[]
[('a1', 'b1', 'c1', 'd2', 'a1', 'b1', 'c2', 'd1'), ('a1', 'b1', 'c2', 'd1', 'a1', 'b1', 'c1', 'd2')]
[]
[('a1',)]
                                        dependencies
5  {"1 => 2": 1.000000, "1 => 3": 0.333333, "2 =>...
[('{"1 => 2": 1.000000, "2 => 1": 1.000000}',), ('{"1 => 2": 1.000000, "2 => 1": 1.000000}',), ('{"1 => 2": 1.000000, "2 => 1": 1.000000}',), ('{"1 => 2": 1.000000, "2 => 1": 1.000000}',), ('{"1 => 2": 1.000000, "2 => 1": 1.000000}',), ('{"1 => 2": 1.000000, "1 => 3": 0.333333, "2 => 1": 1.000000, "2 => 3": 0.333333, "3 => 1": 0.333333, "3 => 2": 0.333333, "1, 2 => 3": 0.333333, "1, 3 => 2": 1.000000, "2, 3 => 1": 1.000000}',)]


In [ ]:
conn.close()